### Install the required packages

In [ ]:
# %pip install python-dotenv
# %pip install requests

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values,load_dotenv

import sys
sys.path.append('../common')
from helper import Helper


query_params = "?api-version=3.0"
# Load the environment variables
load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, "translator/azure/text", api_key, region_override)

### List available languages - commented due to non-unicode result breaking pytest notebook

In [ ]:
# response = helper.get("/languages"+query_params)

# assert response.status_code == 200

# response_json = response.json
# if 'translation' in response_json and response_json['translation']:
#     assert len(response_json['translation']) > 0
# else:
#     raise ValueError("The response does not contain 'translation' property or it has no children.")


### Translate

In [ ]:
body = [{
    "Text": "Hello, how are you?"
}]

response = helper.post("/translate" + query_params + "&to=fr", body)
print(response)
assert response.status_code == 200

response_json = response.json[0]
assert len(response_json) > 0

# Assert that the response contains 'detectLanguage' and 'translation' properties
assert 'detectedLanguage' in response_json, "Response does not contain 'detectedLanguage' property"
assert 'translations' in response_json, "Response does not contain 'translations' property"


### Break Sentence

In [ ]:

body = [{
    "Text": "Hello, how are you?"
}]

response = helper.post("/breaksentence" + query_params , body)

assert response.status_code == 200

response_json = response.json[0]
assert len(response_json) > 0

assert 'detectedLanguage' in response_json, "Response does not contain 'detectedLanguage' property"

### Detect

In [ ]:
body = [{
    "Text": "Hello, how are you?"
}]

response = helper.post("/detect" + query_params, body)

assert response.status_code == 200

response_json = response.json[0]
assert len(response_json) > 0

assert 'isTranslationSupported' in response_json, "Response does not contain 'isTranslationSupported' property"
assert 'isTransliterationSupported' in response_json, "Response does not contain 'isTransliterationSupported' property"
assert 'language' in response_json, "Response does not contain 'language' property"
assert 'score' in response_json, "Response does not contain 'score' property"

### Dictionary Examples

In [ ]:
body = [{"Text":"fly","Translation":"volar"}]

response = helper.post("/dictionary/examples" + query_params + "&from=en&to=fr", body)

assert response.status_code == 200

response_json = response.json[0]
assert len(response_json) > 0

assert 'normalizedSource' in response_json, "Response does not contain 'normalizedSource' property"
assert 'normalizedTarget' in response_json, "Response does not contain 'normalizedTarget' property"
assert 'examples' in response_json, "Response does not contain 'examples' property"

### Dictionary Lookup

In [ ]:
body = [{"Text":"fly"}]

response = helper.post("/dictionary/lookup" + query_params + "&from=en&to=fr", body)

assert response.status_code == 200

response_json = response.json[0]
assert len(response_json) > 0

assert 'normalizedSource' in response_json, "Response does not contain 'normalizedSource' property"
assert 'displaySource' in response_json, "Response does not contain 'displaySource' property"
assert 'translations' in response_json, "Response does not contain 'translations' property"

### Transliterate - testing with latin due to non-unicode result breaking pytest notebook

In [ ]:
body = [{"Text":"salut"}]

response = helper.post("/transliterate" + query_params + "&language=fr&fromScript=latn&toScript=latn", body)

assert response.status_code == 200

response_json = response.json
assert len(response_json) > 0

first_item = response_json[0]
print(first_item)
assert 'text' in first_item, "Response does not contain 'text' property"
assert 'script' in first_item, "Response does not contain 'script' property"